In [57]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import librosa
import os 

In [58]:
# Ajuster les options d'affichage
pd.set_option('display.max_columns', None)  # Aucune limite sur le nombre de colonnes affichées
pd.set_option('display.max_colwidth', None)  # Aucune limite sur la largeur du contenu de la colonne

In [59]:
import csv

input_file_path = "../../Dataset3/CSVs/Audible/val_audible_recording_chunks.csv"
output_file_path = "../../Dataset3/CSVs/Audible/val_audible_recording_chunks2.csv"

with open(input_file_path, 'r', newline='') as infile, open(output_file_path, 'w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    for row in reader:
        updated_row = [cell.replace("Gomphocerippus mollis", "Chorthippus mollis") for cell in row]
        writer.writerow(updated_row)

In [60]:
# Correctly specifying your file path
path_audible_5s_train_csv = "../../Dataset3/CSVs/Audible/train_audible_recording_chunks.csv"

# Using pandas to read the CSV file into a DataFrame
data_train = pd.read_csv(path_audible_5s_train_csv)

C:\Users\cdelm\AppData\Local\Temp\ipykernel_153692\3990417908.py:5: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  data_train = pd.read_csv(path_audible_5s_train_csv)


In [61]:
path_audible_5s_train_audio = "../../Dataset3/Selection morceaux audio 5s/Audible/train/"

In [62]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(data_train['label'])

def clean_data(df):
    labels = df[['label','code_unique','chunk_initial_time','annotation_initial_time','annotation_final_time','sampling_rate']]

    labels["label_str"] = labels["label"]
    # Convertir les étiquettes en nombres
    labels['label'] = label_encoder.transform(labels['label'])

    return labels

data_train = clean_data(data_train)

num_labels = max(data_train['label']) + 1

print(num_labels)

173


C:\Users\cdelm\AppData\Local\Temp\ipykernel_153692\3206858252.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["label_str"] = labels["label"]
C:\Users\cdelm\AppData\Local\Temp\ipykernel_153692\3206858252.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels['label'] = label_encoder.transform(labels['label'])


In [63]:
data_train

,label,code_unique,chunk_initial_time,annotation_initial_time,annotation_final_time,sampling_rate,label_str
0,3,Anura_XXXXigeXXES_01VII2022_335,0,0.453638,5.000000,44100,Alytes obstetricans
1,154,Anura_XXXXigeXXES_01VII2022_335,0,0.000000,5.000000,44100,Tettigonia viridissima
2,3,Anura_XXXXigeXXES_01VII2022_335,5,5.000000,10.000000,44100,Alytes obstetricans
3,154,Anura_XXXXigeXXES_01VII2022_335,5,5.000000,10.000000,44100,Tettigonia viridissima
4,3,Anura_XXXXigeXXES_01VII2022_335,10,10.000000,15.000000,44100,Alytes obstetricans
...,...,...,...,...,...,...,...
12892,27,Aves_KrDe_AwSw00PL_16VII2014_4613,175,175.000000,175.674299,44100,Chrysochraon dispar
12893,141,Aves_KrDe_AwSw00PL_16VII2014_4613,175,175.000000,180.000000,44100,Saxicola rubetra
12894,27,Aves_KrDe_AwSw00PL_16VII2014_4613,180,183.399642,183.863163,44100,Chrysochraon dispar
12895,27,Aves_KrDe_AwSw00PL_16VII2014_4613,180,180.000000,181.313800,44100,Chrysochraon dispar


In [64]:
condition1 = (data_train["annotation_initial_time"] - data_train["chunk_initial_time"] < 3)
print(condition1)

0         True
1         True
2         True
3         True
4         True
         ...  
12892     True
12893     True
12894    False
12895     True
12896     True
Length: 12897, dtype: bool


In [65]:
files_train = os.listdir(path_audible_5s_train_audio)

y_train = []
for file in files_train:
    split_str_file = file.split("_split_")
    split_2 = split_str_file[1].split("_")
    chunk_initial_time = split_2[0]
    code = split_str_file[0]
    condition1 = data_train["code_unique"]==code
    condition2 = data_train["chunk_initial_time"]==int(chunk_initial_time)
    condition_0_3 = (data_train["annotation_initial_time"] - data_train["chunk_initial_time"] < 3)
    condition_3_5 = (data_train["annotation_final_time"] - data_train["chunk_initial_time"] > 3)
    labels_0_3 = data_train[condition1 & condition2 & condition_0_3]["label"].values
    labels_3_5 = data_train[condition1 & condition2 & condition_3_5]["label"].values
    labels_categorical_0_3 = [0] * num_labels
    labels_categorical_3_5 = [0] * num_labels
    for value in labels_0_3:
        labels_categorical_0_3[value] = 1
    for value in labels_3_5:
        labels_categorical_3_5[value] = 1
    y_train.append([labels_categorical_0_3,labels_categorical_3_5])


In [66]:
y_train[0]

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [67]:
liste_labels = []

for i in range(num_labels):
    label_str = data_train[data_train["label"] == i]["label_str"].values[0]
    liste_labels.append(label_str)

print(liste_labels)

['Aegithalos caudatus', 'Alarme', 'Alauda arvensis', 'Alytes obstetricans', 'Anthus trivialis', 'Apis mellifera', 'Avion', 'Bicolorana bicolor', 'Bombina variegata', 'Bos taurus', 'Branche', 'Bug informatique', 'Buteo buteo', 'Camion', 'Canis familiaris', 'Certhia brachydactyla', 'Certhia familiaris', 'Chanson', 'Choc', 'Chorthippus apricarius', 'Chorthippus biguttulus', 'Chorthippus brunneus', 'Chorthippus dorsatus', 'Chorthippus jacobsi', 'Chorthippus mollis', 'Chorthippus saulcyi', 'Chorthippus vagans', 'Chrysochraon dispar', 'Cicada orni', 'Cicadatra atra', 'Cicadetta petryi', 'Cisticola juncidis', 'Cloche', 'Cloche_v', 'Columba palumbus', 'Conocephalus fuscus', 'Corvus corax', 'Corvus corone', 'Cris', 'Curruca communis', 'Curruca melanocephala', 'Cyanistes caeruleus', 'Decticus albifrons', 'Decticus verrucivorus', 'Delichon urbicum', 'Dendrocopos major', 'Disqueuse', 'Déplacement', 'Déplacement animal', 'Eau', 'Emberiza calandra', 'Emberiza cia', 'Emberiza cirlus', 'Emberiza citri

In [68]:
import shutil
from pydub import AudioSegment

trainingFolder = "trainingFolder3/"
dataSourceFolder = path_audible_5s_train_audio
file_num = 0

zero_list = [0] * num_labels
for file in files_train:
    path_source_file = dataSourceFolder + file

    audio = AudioSegment.from_wav(path_source_file)

    # Split the audio into two parts
    part1 = audio[:3000]
    part2 = audio[3000:]

    folderName = ""
    if y_train[file_num][0] != zero_list:
        for i in range(num_labels):
            if y_train[file_num][0][i]:
                folderName = folderName + liste_labels[i] + ","
        folderName = folderName[:-1]
        folderPath = trainingFolder + folderName

        if not(os.path.exists(folderPath)):
            os.makedirs(folderPath)
        
        path_destination_file = folderPath + "/" + file

        part1.export(path_destination_file, format="wav")
    
    folderName = ""
    if y_train[file_num][1] != zero_list:
        for i in range(num_labels):
            if y_train[file_num][1][i]:
                folderName = folderName + liste_labels[i] + ","
        folderName = folderName[:-1]
        folderPath = trainingFolder + folderName

        if not(os.path.exists(folderPath)):
            os.makedirs(folderPath)
        
        path_destination_file = folderPath + "/" + file

        part2.export(path_destination_file, format="wav")

    file_num += 1

In [69]:
def modifFolder(folderName):
    folders = os.listdir(folderName)

    for folder in folders:
        oldName = folderName + folder
        newName = oldName.replace("é", "e")
        os.rename(oldName, newName)

In [70]:
modifFolder(trainingFolder)

In [71]:
# import audio
# import config as cfg

# cfg.setConfig(config)


# string ="trainingFolder\Apis mellifera, Avion, Non identifie, Vespa velutina\Hymenoptera_crcatou31FR_17X2022_819_split_10_15.wav"
# sig, rate = audio.openAudioFile(f, duration=cfg.SIG_LENGTH if cfg.SAMPLE_CROP_MODE == "first" else None, fmin=cfg.BANDPASS_FMIN, fmax=cfg.BANDPASS_FMAX)